In [30]:
import seaborn as sns
import scipy.stats as stats
from scipy import stats
from math import sqrt
import scipy
import numpy as np
import datetime
import pandas as pd 
import sklearn
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.feature_selection import RFECV
import pickle
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression

pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [2]:
# holdout = pd.read_csv('resources/movies_holdout_features.csv', index_col=0)
holdout = pd.read_csv('kc_house_data_test_features.csv')
kc_df = holdout
# zip_score = pd.read_csv('Niche.csv')[['zipcode']]
# kc_df = zip_score.set_index('zipcode').join(kc_df.set_index('zipcode'))

#reading data in
# kc_df = pd.read_csv('kc_house_data_train.csv')
# kc_house_test = pd.read_csv('kc_house_data_test_features.csv')
#zip_score = pd.read_csv('Niche.csv')[['zipcode']]
#kc_df = zip_score.set_index('zipcode').join(kc_house.set_index('zipcode'))

In [3]:
#scaler_file = open('scaler.pickle','rb')
#final_scaler = pickle.load(scaler_file)
model_file = open('model.pickle','rb')
final_model = pickle.load(model_file)
#scaler_file.close()
model_file.close()

In [4]:
final_model.coef_

array([-1.62965636e+04,  2.79506788e+04,  1.54364591e+02,  1.48857442e-01,
       -1.63773505e+04,  3.30342331e+04,  1.83228694e+01, -1.17849561e+04,
        6.93259508e+02,  1.00527465e+02, -7.15557209e+04,  7.76944831e+05,
        2.82828372e+05,  2.57577280e+05,  2.44549855e+05,  2.72619512e+05,
        4.15746498e+04,  1.31725120e+05,  8.20167870e+04,  8.27264071e+04,
       -3.60363752e+04,  1.52590947e+05,  1.64461324e+05,  1.34528313e+05,
        2.16473234e+05,  3.56070953e+05,  2.03694397e+05,  2.60805073e+04,
        1.19855068e+06,  5.01957086e+05,  8.90615685e+04,  2.30380842e+05,
        1.91929589e+05,  4.50236450e+04,  8.98557398e+04,  2.68424501e+04,
        7.34397592e+04,  1.03855348e+05,  1.45717239e+05,  1.70621342e+05,
        1.64041514e+05,  8.02035626e+04, -3.44390921e+04,  5.22726706e+05,
        3.71628695e+05,  5.04494900e+05,  1.36187085e+05,  3.79041389e+05,
        1.30225189e+05,  5.47739973e+05,  6.56450713e+05,  3.52174559e+05,
        3.34288225e+05,  

## Visualations to understand data

In [ ]:
#a understanding of price (mean) vs grade
kc_df.groupby('grade')['price'].mean().plot(kind='bar')
plt.ylabel('price')

In [ ]:
#a graph to understand where all of the price points are located and where the outliers are
fig = plt.figure(figsize=(20,10))
plt.subplot(2,1,1)
plt.ylabel('Frequency')
sns.distplot(kc_df['price'])
plt.subplot(2,1,2)
plt.boxplot(kc_df['price'], vert= False)
plt.xlabel('price')
plt.show()

In [ ]:
#a plot to see the distribution of bedrooms vs how expensive a house is (and see outliers)
plt.scatter(kc_df['price'], kc_df['bedrooms'])
plt.xlabel("Price")
plt.ylabel("Number of Bedrooms")

In [ ]:
# creating a 'heat map' that color codes the correlation between variables and price
price_corr = kc_df[kc_df.columns[:]].corr()['price'][:]
price_corr.sort_values()
price_coef = pd.DataFrame(price_corr)
ax = sns.heatmap(price_coef)

In [5]:
#Getting the index reset
kc_df = kc_df.reset_index()

#dropping the values we don't want to bother us since we know from 
#earlier tests they are not good
kc_df.drop(['id', 'lat', 'long', 'Unnamed: 0', 'date', 'view', 'sqft_living15', 'sqft_lot15' ], axis = 1, inplace = True)



In [6]:
#helps to drop the index from our df
kc_df.drop('index', axis = 1, inplace = True)

In [7]:
kc_df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode
0,4,2.50,2270,11500,1.0,0,3,8,1540,730,1967,0,98034
1,4,2.50,2270,11500,1.0,0,3,8,1540,730,1967,0,98034
2,3,2.50,1470,1779,2.0,0,3,8,1160,310,2005,0,98029
3,3,1.75,1280,16200,1.0,0,3,8,1030,250,1976,0,98077
4,4,2.75,2830,8126,2.0,0,3,8,2830,0,2005,0,98059
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3,2.50,1530,1131,3.0,0,3,8,1530,0,2009,0,98103
4319,4,2.50,2310,5813,2.0,0,3,8,2310,0,2014,0,98146
4320,2,0.75,1020,1350,2.0,0,3,7,1020,0,2009,0,98144
4321,3,2.50,1600,2388,2.0,0,3,8,1600,0,2004,0,98027


## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [ ]:
# def n_sqft_living(row):
#     if row['sqft_living'] > 5000:
#         row['sqft_living'] = 5000    
#     return row

# kc_df = kc_df.apply(n_sqft_living, axis = 1)

In [ ]:
# def n_sqft_lot(row):
#     if row['sqft_lot'] > 75000:
#         row['sqft_lot'] = 75000    
#     return row

# kc_df = kc_df.apply(n_sqft_lot, axis = 1)

In [8]:
kc_df['sqft_liv_lot'] = np.nan

def n_sqft_liv_lot(row):
    row['sqft_liv_lot'] = (row['sqft_living'] / row['sqft_lot'])    
    return row

kc_df = kc_df.apply(n_sqft_liv_lot, axis = 1)

In [9]:
def n_bed(row):
    if row['bedrooms'] ==0:
        row['bedrooms'] = row['floors']
    if row['bedrooms'] > 10:
        row['bedrooms'] = 10
    return row

kc_df = kc_df.apply(n_bed, axis = 1)

In [10]:
def n_bath(row):
    if row['bathrooms'] == 0:
        row['bathrooms'] = 1
    return row

kc_df = kc_df.apply(n_bath, axis = 1)

In [11]:
kc_df['sqft_basement'] = np.where(kc_df['sqft_basement'] != 0, 1, 0)

In [12]:
kc_df['yr_renovate'] = np.nan

def yr_renov(row):
    if row['yr_renovated'] == 0:
        row['yr_renovate'] = 0
    if row['yr_renovated'] != 0:
        row['yr_renovate'] = 2021 - row['yr_built']
    return row 

kc_df = kc_df.apply(yr_renov, axis = 1)

In [13]:
kc_df['yr_updated'] = np.nan

def yr_update(row):
    if row['yr_renovated'] == 0:
        row['yr_updated'] =2021 - row['yr_built']
    elif row['yr_renovated'] != 0:
        row['yr_updated'] = 2021 - row['yr_renovated']
    return row['yr_updated']

kc_df['yr_updated'] = kc_df.apply(yr_update, axis = 1)
kc_df.drop(['yr_built',	'yr_renovated' ], axis = 1, inplace = True)

In [14]:
kc_df['gra_3.0'] = 0

## Setting up Dummies

In [15]:
cat_var = ['zipcode', 'grade', 'waterfront']
cont_var = ['bedrooms', 'bathrooms', 'dist_city', 'sqft_above', 'sqft_living', 'sqft_lot', 'yr_updated']

In [16]:
def to_dummies(df, features):
   
    d_df = []
    for f in features:
        d_df.append(pd.get_dummies(df[f], prefix='{}'.format(str(f)[:3]), drop_first=True))
    df = df.drop(features, axis = 1)
    df = pd.concat([df] + d_df ,axis=1)
    return df

In [17]:
kc_df = to_dummies(kc_df, cat_var)

In [18]:
summary_features = list(kc_df.columns)

In [23]:
kc_df[best_features]

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,sqft_above,sqft_basement,yr_renovate,yr_updated,sqft_liv_lot,zip_98004.0,zip_98005.0,zip_98006.0,zip_98007.0,zip_98008.0,zip_98010.0,zip_98011.0,zip_98014.0,zip_98019.0,zip_98023.0,zip_98024.0,zip_98027.0,zip_98028.0,zip_98029.0,zip_98033.0,zip_98034.0,zip_98038.0,zip_98039.0,zip_98040.0,zip_98045.0,zip_98052.0,zip_98053.0,zip_98055.0,zip_98056.0,zip_98058.0,zip_98059.0,zip_98065.0,zip_98072.0,zip_98074.0,zip_98075.0,zip_98077.0,zip_98092.0,zip_98102.0,zip_98103.0,zip_98105.0,zip_98106.0,zip_98107.0,zip_98108.0,zip_98109.0,zip_98112.0,zip_98115.0,zip_98116.0,zip_98117.0,zip_98118.0,zip_98119.0,zip_98122.0,zip_98125.0,zip_98126.0,zip_98133.0,zip_98136.0,zip_98144.0,zip_98146.0,zip_98148.0,zip_98155.0,zip_98166.0,zip_98168.0,zip_98177.0,zip_98178.0,zip_98188.0,zip_98198.0,zip_98199.0,gra_5.0,gra_6.0,gra_7.0,gra_8.0,gra_9.0,gra_10.0,gra_11.0,gra_12.0,gra_13.0,wat_1.0
0,4.0,2.50,2270.0,11500.0,1.0,3.0,1540.0,1.0,0.0,54.0,0.197391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,4.0,2.50,2270.0,11500.0,1.0,3.0,1540.0,1.0,0.0,54.0,0.197391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,3.0,2.50,1470.0,1779.0,2.0,3.0,1160.0,1.0,0.0,16.0,0.826307,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3.0,1.75,1280.0,16200.0,1.0,3.0,1030.0,1.0,0.0,45.0,0.079012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,4.0,2.75,2830.0,8126.0,2.0,3.0,2830.0,0.0,0.0,16.0,0.348265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3.0,2.50,1530.0,1131.0,3.0,3.0,1530.0,0.0,0.0,12.0,1.352785,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4319,4.0,2.50,2310.0,5813.0,2.0,3.0,2310.0,0.0,0.0,7.0,0.397385,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4320,2.0,0.75,1020.0,1350.0,2.0,3.0,1020.0,0.0,0.0,12.0,0.755556,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4321,3.0,2.50,1600.0,2388.0,2.0,3.0,1600.0,0.0,0.0,17.0,0.670017,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [20]:
len(summary_features)

92

In [21]:
best_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'condition', 'sqft_above', 'sqft_basement', 'yr_renovate', 'yr_updated',
       'sqft_liv_lot', 'zip_98004.0', 'zip_98005.0', 'zip_98006.0',
       'zip_98007.0', 'zip_98008.0', 'zip_98010.0', 'zip_98011.0',
       'zip_98014.0', 'zip_98019.0', 'zip_98023.0', 'zip_98024.0',
       'zip_98027.0', 'zip_98028.0', 'zip_98029.0', 'zip_98033.0',
       'zip_98034.0', 'zip_98038.0', 'zip_98039.0', 'zip_98040.0',
       'zip_98045.0', 'zip_98052.0', 'zip_98053.0', 'zip_98055.0',
       'zip_98056.0', 'zip_98058.0', 'zip_98059.0', 'zip_98065.0',
       'zip_98072.0', 'zip_98074.0', 'zip_98075.0', 'zip_98077.0',
       'zip_98092.0', 'zip_98102.0', 'zip_98103.0', 'zip_98105.0',
       'zip_98106.0', 'zip_98107.0', 'zip_98108.0', 'zip_98109.0',
       'zip_98112.0', 'zip_98115.0', 'zip_98116.0', 'zip_98117.0',
       'zip_98118.0', 'zip_98119.0', 'zip_98122.0', 'zip_98125.0',
       'zip_98126.0', 'zip_98133.0', 'zip_98136.0', 'zip_98144.0',
       'zip_98146.0', 'zip_98148.0', 'zip_98155.0', 'zip_98166.0',
       'zip_98168.0', 'zip_98177.0', 'zip_98178.0', 'zip_98188.0',
       'zip_98198.0', 'zip_98199.0', 'gra_5.0', 'gra_6.0', 'gra_7.0',
       'gra_8.0', 'gra_9.0', 'gra_10.0', 'gra_11.0', 'gra_12.0', 'gra_13.0',
       'wat_1.0']

In [22]:
len(best_features)

82

## Step 3: Predict the holdout set

In [ ]:
#final_scaler = final_scaler.transform(scaler.pickle)
#final_model = final_model.transform(model.pickle)

In [ ]:
transformed_holdout = final_scaler(kc_df)

In [ ]:
X_train.select_dtypes(include=["number"]).columns

In [35]:
final = final_model.predict(kc_df[best_features])

In [36]:
final[:10]

array([549993.06002206, 549993.06002206, 381967.15048381, 267935.74610898,
       517096.21711616, 601002.81607891, 289805.38659748, 290147.91231799,
       414996.73094309, 284612.87557146])

In [40]:
apples = pd.DataFrame(final)

## Step 4: Export your predictions

In [41]:
apples.to_csv('housing_preds_guy.csv')